In [369]:
import numpy as np
import plotly.express as px
np.random.seed(123)
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import warnings 
warnings.filterwarnings('ignore')

import math


import plotly.express as px
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

sns.set_style('darkgrid')
matplotlib.rcParams['font.size'] = 14
matplotlib.rcParams['figure.figsize'] = (10, 6)
matplotlib.rcParams['figure.facecolor'] = '#00000000'

%matplotlib inline


In [370]:
from sklearn.feature_selection import SelectKBest , f_classif , f_regression , VarianceThreshold
from sklearn.model_selection import train_test_split

In [371]:
data = pd.read_csv('POWER_Point_Hourly_20200501_20200731_005d5516N_000d2686W_LST.csv', skiprows = 15)
data.head() # returns the first 5 rows of the dataset

,YEAR,MO,DY,HR,PS,WS10M,WD10M,WS50M,RH2M,PRECTOTCORR,T2M
0,2020,5,1,0,101.05,2.49,264.79,3.32,90.12,0.08,26.15
1,2020,5,1,1,100.97,2.31,260.06,3.05,90.25,0.12,26.02
2,2020,5,1,2,100.92,2.20,254.14,2.85,90.19,0.19,25.91
3,2020,5,1,3,100.90,2.23,250.36,2.88,90.00,0.19,25.87
4,2020,5,1,4,100.92,2.24,247.20,2.88,90.00,0.15,25.83


In [372]:
data.columns # provides a list of all the column names

Index(['YEAR', 'MO', 'DY', 'HR', 'PS', 'WS10M', 'WD10M', 'WS50M', 'RH2M',
       'PRECTOTCORR', 'T2M'],
      dtype='object')

# Linear Model

In [374]:
# For the normalization

def ReScale(data, a):
    data_avg = data.mean()
    scaled_data = a*(data - data_avg)
    return scaled_data 

In [376]:
data_norm=ReScale(data, 0.5)
data_norm['WD10M'] = ReScale(data_norm['WD10M'],0.01)
data_norm.head()

,YEAR,MO,DY,HR,PS,WS10M,WD10M,WS50M,RH2M,PRECTOTCORR,T2M
0,0.0,-0.5,-7.418478,-5.75,0.061791,-0.490478,0.227946,-0.564717,1.27334,-0.089312,0.077097
1,0.0,-0.5,-7.418478,-5.25,0.021791,-0.580478,0.204296,-0.699717,1.33834,-0.069312,0.012097
2,0.0,-0.5,-7.418478,-4.75,-0.003209,-0.635478,0.174696,-0.799717,1.30834,-0.034312,-0.042903
3,0.0,-0.5,-7.418478,-4.25,-0.013209,-0.620478,0.155796,-0.784717,1.21334,-0.034312,-0.062903
4,0.0,-0.5,-7.418478,-3.75,-0.003209,-0.615478,0.139996,-0.784717,1.21334,-0.054312,-0.082903


In [377]:
feature_cols = ['PS','WS10M','WD10M','PRECTOTCORR' , 'T2M']
X = data_norm[feature_cols]

In [378]:
X = data_norm[feature_cols]

In [379]:
X.head()

,PS,WS10M,WD10M,PRECTOTCORR,T2M
0,0.061791,-0.490478,0.227946,-0.089312,0.077097
1,0.021791,-0.580478,0.204296,-0.069312,0.012097
2,-0.003209,-0.635478,0.174696,-0.034312,-0.042903
3,-0.013209,-0.620478,0.155796,-0.034312,-0.062903
4,-0.003209,-0.615478,0.139996,-0.054312,-0.082903


In [380]:
def sliding_windows(data_norm, window_size):#, forecast_over, shift):
    num_batches = len(data_norm) - window_size + 1
    x = []
    for i in range(num_batches):
        _x = data_norm[ i:i+window_size]
        x.append(_x)
    
    #x.append(data[-window_size:,:])
    return np.array(x)

In [381]:
feature_cols = ['PS','WS10M','WD10M','PRECTOTCORR' , 'T2M']
X = data_norm[feature_cols]

In [382]:
shifts =  [X.shift(9) , X.shift(8) ,X.shift(7) ,
           X.shift(6),X.shift(5) , X.shift(4),
           X.shift(3) , X.shift(2), X.shift(1), X]


In [384]:
df1 = pd.concat((shifts) , axis = 1).dropna(how = 'any')
df1.head()

,PS,WS10M,WD10M,PRECTOTCORR,T2M,PS,WS10M,WD10M,PRECTOTCORR,T2M,...,PS,WS10M,WD10M,PRECTOTCORR,T2M,PS,WS10M,WD10M,PRECTOTCORR,T2M
9,0.061791,-0.490478,0.227946,-0.089312,0.077097,0.021791,-0.580478,0.204296,-0.069312,0.012097,...,0.096791,-0.645478,0.006446,-0.099312,0.947097,0.096791,-0.530478,-0.044854,-0.104312,1.352097
10,0.021791,-0.580478,0.204296,-0.069312,0.012097,-0.003209,-0.635478,0.174696,-0.034312,-0.042903,...,0.096791,-0.530478,-0.044854,-0.104312,1.352097,0.081791,-0.415478,-0.089354,-0.109312,1.647097
11,-0.003209,-0.635478,0.174696,-0.034312,-0.042903,-0.013209,-0.620478,0.155796,-0.034312,-0.062903,...,0.081791,-0.415478,-0.089354,-0.109312,1.647097,0.056791,-0.235478,-0.139004,-0.104312,1.847097
12,-0.013209,-0.620478,0.155796,-0.034312,-0.062903,-0.003209,-0.615478,0.139996,-0.054312,-0.082903,...,0.056791,-0.235478,-0.139004,-0.104312,1.847097,0.021791,-0.020478,-0.180354,-0.089312,1.922097
13,-0.003209,-0.615478,0.139996,-0.054312,-0.082903,0.021791,-0.635478,0.128896,-0.074312,-0.082903,...,0.021791,-0.020478,-0.180354,-0.089312,1.922097,-0.018209,0.154522,-0.213754,-0.069312,1.892097


In [385]:
new_cols = []
for i in range(1, 50):
    new_cols.append('feat_'+ str(i))

new_cols  += ['T2M']
df1.columns = new_cols

In [386]:
df1.head()

,feat_1,feat_2,feat_3,feat_4,feat_5,feat_6,feat_7,feat_8,feat_9,feat_10,...,feat_41,feat_42,feat_43,feat_44,feat_45,feat_46,feat_47,feat_48,feat_49,T2M
9,0.061791,-0.490478,0.227946,-0.089312,0.077097,0.021791,-0.580478,0.204296,-0.069312,0.012097,...,0.096791,-0.645478,0.006446,-0.099312,0.947097,0.096791,-0.530478,-0.044854,-0.104312,1.352097
10,0.021791,-0.580478,0.204296,-0.069312,0.012097,-0.003209,-0.635478,0.174696,-0.034312,-0.042903,...,0.096791,-0.530478,-0.044854,-0.104312,1.352097,0.081791,-0.415478,-0.089354,-0.109312,1.647097
11,-0.003209,-0.635478,0.174696,-0.034312,-0.042903,-0.013209,-0.620478,0.155796,-0.034312,-0.062903,...,0.081791,-0.415478,-0.089354,-0.109312,1.647097,0.056791,-0.235478,-0.139004,-0.104312,1.847097
12,-0.013209,-0.620478,0.155796,-0.034312,-0.062903,-0.003209,-0.615478,0.139996,-0.054312,-0.082903,...,0.056791,-0.235478,-0.139004,-0.104312,1.847097,0.021791,-0.020478,-0.180354,-0.089312,1.922097
13,-0.003209,-0.615478,0.139996,-0.054312,-0.082903,0.021791,-0.635478,0.128896,-0.074312,-0.082903,...,0.021791,-0.020478,-0.180354,-0.089312,1.922097,-0.018209,0.154522,-0.213754,-0.069312,1.892097


In [387]:
X= df1.drop('T2M' ,axis = 1)

In [388]:
y = df1['T2M']
X= df1.drop('T2M' ,axis = 1)


from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size =0.25 ,   random_state=30)

In [389]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(1649, 49)
(1649,)
(550, 49)
(550,)


In [390]:
from sklearn.linear_model import LinearRegression

# instantiate the model
linreg = LinearRegression()

# fit the model to the training data (learn the coefficients)
#linreg.compile(loss='mean_squared_error', optimizer='sgd')
#nb_epoch = 20
linreg.fit(X_train, y_train)

LinearRegression()

In [391]:
# print the intercept and coefficients
print(linreg.intercept_)
print(linreg.coef_)

-0.00045154871854356966
[-4.32155555e-01  4.28693692e-02  6.41359410e-04 -7.33405814e-03
  2.25865750e-01  1.07581233e+00 -9.14683093e-02 -2.90554146e-03
 -8.14574614e-03 -2.85385539e-01  2.40936807e-01  7.72685419e-02
 -3.60976370e-03  4.71590257e-02  6.64197099e-02 -8.90179187e-01
 -2.94955984e-02  3.33709308e-03 -8.08371305e-02 -4.31621380e-02
  6.97494000e-03 -2.84031233e-02 -1.68025877e-02  7.10218705e-02
  6.40180688e-02 -4.83006246e-01 -4.17667535e-02  2.20089680e-02
 -3.20244446e-04 -6.24016891e-02  9.06175920e-02  1.11498559e-01
 -1.35033419e-02 -5.09876873e-02  2.29657587e-01  1.23684586e+00
  3.40050552e-02 -5.02294409e-02  3.32813487e-02 -1.20328735e+00
 -1.82153509e+00 -2.08154634e-01  6.31658090e-02  5.58746334e-03
  1.99908399e+00  9.62118052e-01  1.24751045e-01  4.55730300e-02
 -4.68831477e-03]


In [392]:
y_train_pred=linreg.predict(X_train)
y_train_pred

array([-0.18304921,  0.62494474,  0.29608319, ...,  0.16050045,
        0.56660268,  0.01294907])

In [393]:
y_pred = linreg.predict(X_test)

In [395]:
df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
df.head()

,Actual,Predicted
2149,-0.282903,-0.287396
999,-0.042903,-0.082434
1074,0.007097,0.034190
1940,-1.097903,-1.077956
660,1.202097,1.181534


In [396]:
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))  
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))  
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

Mean Absolute Error: 0.03269545773176973
Mean Squared Error: 0.002243574918215259
Root Mean Squared Error: 0.04736639017505196
